<a href="https://colab.research.google.com/github/pandhare-sneha/sneha/blob/master/Banking_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install Apache Spark 3.0.1 with Hadoop 2.7 from here.
!wget https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz

# Now, we just need to unzip that folder.
!tar -xvzf spark-3.0.0-bin-hadoop2.7.tgz
!pip install findspark


import os
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"
import findspark
findspark.init()

--2023-12-19 12:10:16--  https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 220272364 (210M) [application/x-gzip]
Saving to: ‘spark-3.0.0-bin-hadoop2.7.tgz.1’

spark-3.0.0-bin-had 100%[===================>] 210.07M  22.2MB/s    in 10s     

2023-12-19 12:10:27 (20.3 MB/s) - ‘spark-3.0.0-bin-hadoop2.7.tgz.1’ saved [220272364/220272364]

spark-3.0.0-bin-hadoop2.7/
spark-3.0.0-bin-hadoop2.7/NOTICE
spark-3.0.0-bin-hadoop2.7/kubernetes/
spark-3.0.0-bin-hadoop2.7/kubernetes/tests/
spark-3.0.0-bin-hadoop2.7/kubernetes/tests/worker_memory_check.py
spark-3.0.0-bin-hadoop2.7/kubernetes/tests/py_container_checks.py
spark-3.0.0-bin-hadoop2.7/kubernetes/tests/pyfiles.py
spark-3.0.0-bin-hadoop2.7/kubernetes/dockerfiles/
spark-3.0.0-bin-hadoop2

# New Section

Customers Sqlite Reading in datafram

In [ ]:
import sqlite3

# Replace 'your_database.db' with the actual name of your SQLite database file
db_path = '/Customers'

# Connect to SQLite database
conn = sqlite3.connect(db_path)

# Create a cursor to execute SQL commands
cursor = conn.cursor()

# Example: Execute a query
cursor.execute("SELECT * FROM customers")
rows = cursor.fetchall()

# Display the results
for row in rows:
    print(row)

# Close the cursor and connection


(1, 'John', 'Doe', '1980-05-15')
(2, 'Jane', 'Smith', '1992-08-21')
(3, 'Alice', 'Johnson', '1975-02-10')
(4, 'Sarah', 'Jones', '1988-12-03')
(5, 'David', 'Brown', '1995-04-18')
(6, 'Emma', 'Miller', '1982-07-25')


In [217]:
import pandas as pd

# Example: Read a table into a DataFrame
df = pd.read_sql_query("SELECT * FROM customers", conn)

# Display the DataFrame
# print(df)

spark_df = spark.createDataFrame(df)

spark_df.show()


/content/spark-3.0.0-bin-hadoop2.7/python/pyspark/sql/pandas/conversion.py:327: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+-----------+----------+---------+-------------+
|customer_id|first_name|last_name|date_of_birth|
+-----------+----------+---------+-------------+
|          1|      John|      Doe|   1980-05-15|
|          2|      Jane|    Smith|   1992-08-21|
|          3|     Alice|  Johnson|   1975-02-10|
|          4|     Sarah|    Jones|   1988-12-03|
|          5|     David|    Brown|   1995-04-18|
|          6|      Emma|   Miller|   1982-07-25|
+-----------+----------+---------+-------------+



**Write a spark dataframe to show the balance amount for an account_id = 1:**


In [ ]:
import json
import findspark
findspark.init()
from pyspark.sql import SparkSession

json_file_path = '/content/accounts.json'

# Parse JSON data from file to Python dictionary
with open(json_file_path, 'r') as file:
    parsed_data = json.load(file)

lst = []
for i in parsed_data['accounts']:
    lst.append(i)

# print(lst)

spark = SparkSession.builder.appName("accounts").getOrCreate()
accounts_df = spark.createDataFrame(lst)


accounts_df.select("balance").filter(col("account_id")==1).show()


/content/spark-3.0.0-bin-hadoop2.7/python/pyspark/sql/session.py:378: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+-------+
|balance|
+-------+
|   5000|
+-------+



**`Reading transactions in dataframe`**

In [220]:
df_transactions = spark.read.csv("/content/transactions.csv",inferSchema=True,header=True)
df_transactions.show()

+--------------+----------+----------------+------+-------------------+
|transaction_id|account_id|transaction_type|amount|   transaction_date|
+--------------+----------+----------------+------+-------------------+
|             1|         1|         Deposit|  1000|2023-01-15 08:30:00|
|             2|         1|      Withdrawal|   500|2023-02-02 12:45:00|
|             3|         2|         Deposit|  2000|2023-03-10 15:20:00|
|             4|         3|      Withdrawal|  1000|2023-04-05 10:10:00|
|             5|         4|         Deposit|  1500|2023-05-20 09:00:00|
|             6|         5|         Deposit|  2000|2023-06-12 11:30:00|
|             7|         3|      Withdrawal|   800|2023-07-08 14:15:00|
|             8|         2|         Deposit|  3000|2023-08-22 16:45:00|
|             9|         4|      Withdrawal|  1500|2023-09-14 09:30:00|
|            10|         1|         Deposit|  1200|2023-10-01 10:00:00|
+--------------+----------+----------------+------+-------------

**List Transactions for an account_id = 1:**

In [ ]:
df.filter(col("account_id")==1).show()

+--------------+----------+----------------+------+-------------------+
|transaction_id|account_id|transaction_type|amount|   transaction_date|
+--------------+----------+----------------+------+-------------------+
|             1|         1|         Deposit|  1000|2023-01-15 08:30:00|
|             2|         1|      Withdrawal|   500|2023-02-02 12:45:00|
|            10|         1|         Deposit|  1200|2023-10-01 10:00:00|
+--------------+----------+----------------+------+-------------------+



List Accounts with a zero balance:


In [ ]:
accounts_df.select("*").filter(col("balance")==0).show()


+----------+------------+-------+-----------+-----------+
|account_id|account_type|balance|customer_id|employee_id|
+----------+------------+-------+-----------+-----------+
+----------+------------+-------+-----------+-----------+



**Find the Oldest Customer:**

In [ ]:
oldest_customer = spark_df.orderBy(col("date_of_birth")).first()

# Show the oldest customer
print("\nOldest Customer:")
print(oldest_customer)


Oldest Customer:
Row(customer_id=3, first_name='Alice', last_name='Johnson', date_of_birth='1975-02-10')


Calculate the Total Interest Earned Across All Accounts:


In [ ]:
interest_rate = 0.00

# Calculate interest for each account
interest_df = accounts_df.withColumn("interest_earned", col("balance") * interest_rate)

# Show the DataFrame with interest earned
print("DataFrame with Interest Earned:")
interest_df.show()

# Calculate the total interest earned across all accounts
total_interest_earned = interest_df.select("interest_earned").agg({"interest_earned": "sum"}).collect()[0][0]

# Show the total interest earned
print("\nTotal Interest Earned Across All Accounts:", total_interest_earned)

DataFrame with Interest Earned:
+----------+------------+-------+-----------+-----------+---------------+
|account_id|account_type|balance|customer_id|employee_id|interest_earned|
+----------+------------+-------+-----------+-----------+---------------+
|         1|     Savings|   5000|          1|          2|            0.0|
|         2|    Checking|   1000|          1|          3|            0.0|
|         3|     Savings|   8000|          2|          4|            0.0|
|         4|    Checking|   3000|          3|          1|            0.0|
|         5|    Checking|   2500|          2|          2|            0.0|
|         6|     Savings|   6000|          3|          3|            0.0|
|         7|    Checking|  12000|          4|          4|            0.0|
|         8|     Savings|   3000|          5|          2|            0.0|
+----------+------------+-------+-----------+-----------+---------------+


Total Interest Earned Across All Accounts: 0.0


**List All Accounts with Customer Information:**

In [ ]:


all_accounts_info = accounts_df.join(spark_df,'customer_id')
selected_columns = all_accounts_info.select("customer_id", "first_name", "last_name", "account_id", "account_type", "balance").orderBy("account_id")


In [ ]:
selected_columns.show()

+-----------+----------+---------+----------+------------+-------+
|customer_id|first_name|last_name|account_id|account_type|balance|
+-----------+----------+---------+----------+------------+-------+
|          1|      John|      Doe|         1|     Savings|   5000|
|          1|      John|      Doe|         2|    Checking|   1000|
|          2|      Jane|    Smith|         3|     Savings|   8000|
|          3|     Alice|  Johnson|         4|    Checking|   3000|
|          2|      Jane|    Smith|         5|    Checking|   2500|
|          3|     Alice|  Johnson|         6|     Savings|   6000|
|          4|     Sarah|    Jones|         7|    Checking|  12000|
|          5|     David|    Brown|         8|     Savings|   3000|
+-----------+----------+---------+----------+------------+-------+



In [ ]:
all_accounts_info = accounts_df.join(spark_df,'customer_id')
selected_columns = all_accounts_info.select("customer_id", "first_name", "last_name", "account_id", "account_type", "balance").groupBy("account_id")

Calculate Total Balance for Each Customer:


In [ ]:
from pyspark.sql.functions import sum, col

# Calculate Total Balance for Each Customer

# Ensure the "balance" column is cast to integer
all_accounts_info = all_accounts_info.withColumn("balance", col("balance").cast("int"))

total_balance_per_customer = (
    all_accounts_info
    .groupBy("customer_id", "first_name", "last_name")
    .agg(sum(col("balance")).alias("total_balance"))
)



In [ ]:
total_balance_per_customer.show()

+-----------+----------+---------+-------------+
|customer_id|first_name|last_name|total_balance|
+-----------+----------+---------+-------------+
|          5|     David|    Brown|         3000|
|          1|      John|      Doe|         6000|
|          3|     Alice|  Johnson|         9000|
|          2|      Jane|    Smith|        10500|
|          4|     Sarah|    Jones|        12000|
+-----------+----------+---------+-------------+



**Find Customers with Multiple Accounts:**

In [ ]:
from pyspark.sql.functions import countDistinct, col

# Ensure the "balance" column is cast to integer
all_accounts_info = all_accounts_info.withColumn("balance", col("balance").cast("int"))

# Count the number of accounts for each customer
customer_account_count = (
    all_accounts_info
    .groupBy("customer_id",)
    .agg(countDistinct("account_id").alias("account_count"))
)


In [ ]:
accounts_df.show()

+----------+------------+-------+-----------+-----------+
|account_id|account_type|balance|customer_id|employee_id|
+----------+------------+-------+-----------+-----------+
|         1|     Savings|   5000|          1|          2|
|         2|    Checking|   1000|          1|          3|
|         3|     Savings|   8000|          2|          4|
|         4|    Checking|   3000|          3|          1|
|         5|    Checking|   2500|          2|          2|
|         6|     Savings|   6000|          3|          3|
|         7|    Checking|  12000|          4|          4|
|         8|     Savings|   3000|          5|          2|
+----------+------------+-------+-----------+-----------+



In [ ]:
customer_account_count.show()

+-----------+-------------+
|customer_id|account_count|
+-----------+-------------+
|          5|            1|
|          1|            2|
|          3|            2|
|          2|            2|
|          4|            1|
+-----------+-------------+



In [224]:
new_cus= spark_df.select("customer_id","first_name","last_name").join(accounts_df.select("account_id","customer_id","account_type","balance"),on = "customer_id")
new_cus.show()

+-----------+----------+---------+----------+------------+-------+
|customer_id|first_name|last_name|account_id|account_type|balance|
+-----------+----------+---------+----------+------------+-------+
|          5|     David|    Brown|         8|     Savings|   3000|
|          1|      John|      Doe|         1|     Savings|   5000|
|          1|      John|      Doe|         2|    Checking|   1000|
|          3|     Alice|  Johnson|         4|    Checking|   3000|
|          3|     Alice|  Johnson|         6|     Savings|   6000|
|          2|      Jane|    Smith|         3|     Savings|   8000|
|          2|      Jane|    Smith|         5|    Checking|   2500|
|          4|     Sarah|    Jones|         7|    Checking|  12000|
+-----------+----------+---------+----------+------------+-------+



List Transction with account and customer Inforamation

In [226]:
customer_transction = df_transactions.select("transaction_id","transaction_type","amount","transaction_date","account_id").\
join(new_cus.select("customer_id","first_name","last_name","account_id","account_type"), on = "account_id")
customer_transction.show()

+----------+--------------+----------------+------+-------------------+-----------+----------+---------+------------+
|account_id|transaction_id|transaction_type|amount|   transaction_date|customer_id|first_name|last_name|account_type|
+----------+--------------+----------------+------+-------------------+-----------+----------+---------+------------+
|         1|            10|         Deposit|  1200|2023-10-01 10:00:00|          1|      John|      Doe|     Savings|
|         1|             2|      Withdrawal|   500|2023-02-02 12:45:00|          1|      John|      Doe|     Savings|
|         1|             1|         Deposit|  1000|2023-01-15 08:30:00|          1|      John|      Doe|     Savings|
|         2|             8|         Deposit|  3000|2023-08-22 16:45:00|          1|      John|      Doe|    Checking|
|         2|             3|         Deposit|  2000|2023-03-10 15:20:00|          1|      John|      Doe|    Checking|
|         4|             9|      Withdrawal|  1500|2023-

**Calculate Average Transaction Amount:**

In [231]:
from pyspark.sql.functions import col, avg
avg_tranasction = customer_transction.agg(avg("amount").alias("Avg_transction_amount"))
avg_tranasction.show()

+---------------------+
|Avg_transction_amount|
+---------------------+
|               1450.0|
+---------------------+



**Identify High-Value Customers with Total Balance:**

In [232]:
high_value_cust = total_balance_per_customer.filter(col("total_balance")>10000)
high_value_cust.show()

+-----------+----------+---------+-------------+
|customer_id|first_name|last_name|total_balance|
+-----------+----------+---------+-------------+
|          2|      Jane|    Smith|        10500|
|          4|     Sarah|    Jones|        12000|
+-----------+----------+---------+-------------+



**List Employees and Their Assigned Customers:**

In [250]:
from pyspark.sql import SparkSession
from lxml import etree


# Define the path to the XML file
xml_file_path = "/content/employees.xml"

# Parse XML data using lxml
xml_parser = etree.XMLParser(recover=True)
tree = etree.parse(xml_file_path, parser=xml_parser)

# Extract data from XML and create a list of dictionaries
data = []
for data_record in tree.xpath("//DATA_RECORD"):
    record_data = {}
    for element in data_record.iterchildren():
        record_data[element.tag] = element.text
    data.append(record_data)

# Create PySpark DataFrame from the list of dictionaries
df_emp = spark.createDataFrame(data)

# Show the DataFrame
df_emp.show()





/content/spark-3.0.0-bin-hadoop2.7/python/pyspark/sql/session.py:378: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+---------+-----------+----------+---------+--------+
|branch_id|employee_id|first_name|last_name|position|
+---------+-----------+----------+---------+--------+
|        2|          1|      Mike|  Johnson| Manager|
|        3|          2|     Emily| Williams|  Teller|
|        2|          3|    Robert|    Davis|  Teller|
|        3|          4|    Olivia|   Wilson|  Teller|
|        2|          5|    Daniel|  Johnson| Analyst|
|        3|          6|    Sophia|    Clark| Manager|
|        2|          7|      Mike|  Johnson| Manager|
|        3|          8|     Emily| Williams|  Teller|
|        2|          9|    Robert|    Davis|  Teller|
|        3|         10|    Olivia|   Wilson|  Teller|
|        2|         11|    Daniel|  Johnson| Analyst|
|        3|         12|    Sophia|    Clark| Manager|
+---------+-----------+----------+---------+--------+



In [252]:
assign_customer = df_emp.select("employee_id","first_name","last_name","position").\
join(accounts_df.select("customer_id", "employee_id"),"employee_id","left").\
join(spark_df.select("customer_id",col("first_name").alias("customer_first_name"),col("last_name").\
                     alias("customer_first_name")),"customer_id","left")

assign_customer.show()

+-----------+-----------+----------+---------+--------+-------------------+-------------------+
|customer_id|employee_id|first_name|last_name|position|customer_first_name|customer_first_name|
+-----------+-----------+----------+---------+--------+-------------------+-------------------+
|       null|          7|      Mike|  Johnson| Manager|               null|               null|
|       null|          6|    Sophia|    Clark| Manager|               null|               null|
|       null|          9|    Robert|    Davis|  Teller|               null|               null|
|       null|          5|    Daniel|  Johnson| Analyst|               null|               null|
|       null|         10|    Olivia|   Wilson|  Teller|               null|               null|
|       null|         12|    Sophia|    Clark| Manager|               null|               null|
|       null|          8|     Emily| Williams|  Teller|               null|               null|
|       null|         11|    Daniel|  Jo

**5.	Calculate the Total Number of Transactions for Each Account Type:**

In [255]:
from pyspark.sql.functions import count

transction_count = customer_transction.groupBy("account_type").agg(count("*").alias("num_trnasction"))
transction_count.show()

+------------+--------------+
|account_type|num_trnasction|
+------------+--------------+
|     Savings|             5|
|    Checking|             5|
+------------+--------------+



** 6. Find Customers with No Accounts:
Note Que 6 and 7 are same **

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

latest_transction  =

7.	Find Customers with No Accounts

In [258]:
customer_with_no_account = spark_df.join(accounts_df,"customer_id","left_anti").show()

+-----------+----------+---------+-------------+
|customer_id|first_name|last_name|date_of_birth|
+-----------+----------+---------+-------------+
|          6|      Emma|   Miller|   1982-07-25|
+-----------+----------+---------+-------------+



**9.	Calculate the Total Withdrawals for Each Customer:**

In [269]:
from pyspark.sql import functions as F
Withdrawals = df_transactions.filter(df_transactions["transaction_type"]=="Withdrawal")

total_Withdrawals  = Withdrawals.groupBy("account_id").agg(F.sum("amount").alias("total_widrwal")).show()

+----------+-------------+
|account_id|total_widrwal|
+----------+-------------+
|         1|          500|
|         3|         1800|
|         4|         1500|
+----------+-------------+



**10.	Find Duplicate Transactions:**

In [275]:
from pyspark.sql.window import Window

data_partition = Window.partitionBy("customer_id","amount","transaction_type","transaction_date")
transactions_count = customer_transction.withColumn("transaction_count",F.count("*").over(data_partition))

duplicate_transction  =transactions_count.filter("transaction_count">1)
result = duplicate_transction.select("transction_id","transaction_count")

TypeError: ignored